# Comparing RandomSelect (RSelect) with DeterministicSelect (DSelect)

**Statement of Problem:**
- Input array `A` with `n` distinct (for simplicity) numbers and a number `i` in {1,2,...,n}.
- Output the `i`-th order statistic of the array `A`.

## RSelect

Like the quicksort algorithm, the RSelect algorithm works in-place (no copies of any arrays are made). Furthermore, the workhorse of the RSelect algorithm is also the partition procedure which was introduced for quicksort. As partition utilizes a random selection of a "good pivot", the average running time of quicksort was shown to be O(nlogn). Similarly, we can show that RSelect runs, on average, in linear time (i.e., O(n)). *The fact that it runs in linear time also implies that the Selection problem is an easier problem then the Sorting problem (i.e., the best we can do there is a running time of O(nlogn).*

In [210]:
def partition(A, l=None, r=None):
    """
    input array is A[a_l,...,a_r] where a_l is the pivot
    """
    if l == None:
        l = 0
    if r == None:
        r = len(A)-1
    # pivot WLOG is considered to be the first element
    p = A[l]
    i = l + 1
    for j in range(l+1,r+1):
        if A[j] < p: # otherwise do nothing and increase pointer j
            # swap A[i] with A[j]
            A[i], A[j] = A[j], A[i]
            i += 1
    # finally swap A[i-1] with A[l] to place pivot in correct position
    A[i-1], A[l] = A[l], A[i-1]
    return i-1 # returns index of the pivot

In [211]:
def choosePivot(A,l=None,r=None):
    """
    This function will choose a pivot for the considered
    subset of A = [a_l,...,a_r]. It also does the 
    swap to put the pivot in the ell-th position. This is 
    required because the parition function assumes that
    the pivot element is located in the first element of
    the subset A.
    """
    if l == None:
        l = 0
    if r == None:
        r = len(A) - 1
    p = np.random.randint(l,r+1, size = 1)[0]
    # pre-process A now, so that the pivot is the first element of array 
    A[l],A[p] = A[p],A[l]
    return p

With the partition and choosePivot functions from the Quicksort algorithm, we are now ready to present the RSelect algorithm.

In [212]:
def RSelect(A,i, l=None, r=None):
    """
    Input array=A, orderstat=i, A=[a_l,...,a_r] where l,r take values in (0,1,...,len(A)-1)
    Swaps are done in place via paritition and choosePivot functions
    """
    if r == None:
        r = len(A)-1 # index of last element in A
    if l == None:
        l = 0 # index of first element in A
    
    n = r-l+1
    if n == 1:
        return A[l] 
        """
        because we are swapping in place, 
        the median will end up in ell-th position 
        when recursions reduce down to base case
        """
    else:
        p = choosePivot(A,l,r)
        ix = partition(A,l,r) # gives the index of the pivot after partition
        # A[l:r] = [a_l,...a_ix,...a_r] 
        j = ix - l + 1 # the pivot is actually the j-th order statistic of the subarray A[l:r] where j=ix-l+1 
        #print 'i:',i,'j:',j, 'ix:',ix, A[l:r+1]
        if j == i:
            return A[ix]
        elif j > i:
            return RSelect(A,i,l,ix-1)
        else:
            return RSelect(A,i-j,ix+1,r)    

In [213]:
import numpy as np

In [224]:
A = list(np.random.permutation(10)+1)

In [225]:
A

[4, 3, 10, 9, 1, 7, 2, 5, 8, 6]

In [226]:
RSelect(A,3)

3

In [227]:
# done in place
A

[1, 2, 3, 4, 10, 7, 9, 5, 8, 6]

In [228]:
RSelect(A,9)

9

In [229]:
A

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [238]:
A = [123,456,789,101,121,131,141]

In [239]:
RSelect(A,2)

121

In [240]:
A

[101, 121, 123, 131, 789, 456, 141]

Note that RSelect continues to recursively call itself until either the randomly chosen pivot is equal to the j-th order statistic of A[l:r] or the problem reduces down to the base case.

## DSelect 

The body of the algorithm in the Deterministic version of this select algorithm is identical to that of Rselect, with the only exception being the way in which we choose the Pivot. Recall that in RSelect, we choose the pivot randomly based on a uniform distribution. In contrast, in DSelect we will choose this pivot deterministically. 

It is obvious, that the best pivot to choose the median as it will give us the most balanced recursive procedure (but this of course is a circular argument). **The big idea of the deterministic ChoosePivot is the "median of medians".**

The Deterministic ChoosePivot:

1. Break A into groups of 5, sort each group (use any sorting method...mergeSort or Quicksort, won't matter as we are only sorting 5 elements!)
2. Populate an array C with the n/5 "middle elements" of each group
3. Use a recursive call to choose our pivot (i.e., recursively find the median of C). So, p=Select(C,n/5,n/10)

- *Tim Roughgarden uses the analogy of tournament of teams, whose winning teams from each of the 5 groups represents the middle elements, and the final champion is the middle element of these 1st round winners.*

- *After choosing the pivot in this fashion, the DSelect follows the same algorithmic procedure as RSelect*

Some notes on DSelect:
- This is algorithm does not operate in place. So DSelect is not as fast RSelect
- Running time is O(n) (in worst case)

Will use merge_sort since it does not affect the array A

In [152]:
def combine(x,y):
    z = []
    # having nx and ny allows for combinin
    nx = len(x)
    ny = len(y) # my merge_sort function will always send y as the "bigger half" in case of uneveness
    # starting positions for our pointers
    i = 0
    j = 0
    while i < nx and j < ny:
        if x[i] < y[j]:
            z.append(x[i])
            i += 1
        else:
            z.append(y[j])
            j += 1
    # append the remaining symbols
    if j <= i: # there are elements remaining in y (include = because j can equal to nx but still has one element left)
        for el in y[j:]:
            z.append(el)
    else: # there are elements in remaining in x
        for el in x[i:]:
            z.append(el)
    return z

def merge_sort(x):
    n = len(x)
    half = int(n/2)
    if n == 1:
        return [x[0]] # returns a list in basic case
    else:
        foo = merge_sort(x[:half])
        bar = merge_sort(x[half:])
        return combine(foo,bar)

In [153]:
from __future__ import division

In [276]:
def firstrdmedians(A):
    n = len(A) 
    m = int(n/5)
    rem = n % 5
    C = []
    for i in range(m):
        C.append(merge_sort(A[i*5:(i+1)*5])[2])
    if rem != 0:
        if rem % 2 == 0 :
            mid = int(rem/2)
        else:
            mid = int((rem + 1)/2)
        C.append(merge_sort(A[-rem:])[mid-1])
    return C

In [282]:
firstrdmedians([3,2,1])

[2]

In [396]:
def DSelect(A,i):
    assert i>= 1 and i <= len(A)
    n = len(A)
    if n <= 1: # base cases are any arrays with elements less than 5
        return A[0]
    else:        
        C = firstrdmedians(A)
        mid = int(len(C)/2)
        if len(C) == 1:
            p = C[0]
        else:
            p = DSelect(C, mid)
        # swap pivot postion into first position of A before calling on partition procedure
        foo = A.index(p)
        A[0],A[foo] = A[foo],A[0]
        #print 'A:',A, 'p:',p
        ix = partition(A)
        #print 'A after partition:', A
        # the pivot is the j-th order statistic of A where j=ix+1 (ix is index of pivot)
        j = ix + 1
        #print 'index of p:',ix, 'pivot is jth:',j
        if j == i:
            return p
        elif j > i:
            firstPartA = A[:ix]
            return DSelect(firstPartA,i)
        else:
            secondPartA = A[ix+1:]
            return DSelect(secondPartA,i-j) 

In [402]:
A=list(np.random.permutation(12)+1)
A

[9, 7, 1, 2, 11, 8, 12, 4, 6, 3, 10, 5]

In [403]:
DSelect(A,6)

6

In [404]:
# A is rearranged a bit due to the partition procedure after finding pivot
A

[3, 1, 2, 4, 5, 8, 12, 7, 6, 11, 10, 9]

In [405]:
A = [123,456,789,101,121,131,141]

In [406]:
DSelect(A,2)

121

In [407]:
# DSelect does not operate in place!
A

[121, 101, 123, 456, 789, 131, 141]

## RSelect vs. DSelect on selecting a random order statistic of random permutations of lists [1..10^i], for i=0,1,2,...8 

In [410]:
import copy
import time

In [411]:
# creating a dicts of arrays to be sorted
t0=time.time()
permutations = {i:list(np.random.permutation(10**i)+1) for i in range(9)}
t1=time.time()
t1-t0

34.202986001968384

In [412]:
permutations_copy1 = {key:list[:] for key,list in permutations.items()}
permutations_copy2 = {key:list[:] for key,list in permutations.items()}

In [422]:
np.random.randint(1,11,size=1)[0]

7

In [430]:
# random ith order stats
orders = [np.random.randint(10**i,size=1)[0]+1 for i in range(9)]
orders

[1, 5, 20, 410, 3166, 30119, 357572, 2436981, 92389754]

In [433]:
# testing RSelect 
times = {}
for i in range(9):
    t0=time.time()
    RSelect(permutations_copy1[i],orders[i])
    t1=time.time()
    times[i] = t1-t0
times

{0: 4.0531158447265625e-06,
 1: 0.00014090538024902344,
 2: 0.00014710426330566406,
 3: 0.0023679733276367188,
 4: 0.006084918975830078,
 5: 0.06109189987182617,
 6: 0.5298070907592773,
 7: 6.315201044082642,
 8: 97.2131609916687}

In [434]:
# testing DSelect 
times = {}
for i in range(9):
    t0=time.time()
    DSelect(permutations_copy2[i],orders[i])
    t1=time.time()
    times[i] = t1-t0
times

{0: 4.57763671875e-05,
 1: 0.00025582313537597656,
 2: 0.0017750263214111328,
 3: 0.0159299373626709,
 4: 0.11660289764404297,
 5: 0.9828341007232666,
 6: 9.664698839187622,
 7: 97.22481513023376,
 8: 1061.6915938854218}

As Tim Roughgarden claimed, DSelect is not as fast as RSelect because it does not operate in place.